In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers

In [ ]:
# загружаем 5000 позитивных и негативных твитов

df_tweets = pd.read_csv('train_data.csv', sep='\t')
df_tweets = df_tweets.dropna()

In [ ]:
df_tweets.head(2)

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,label
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",спортсмена,PROFESSION,86,96,0
1,«За всю нашу долгую карьеру нам довелось играт...,музыкантов,PROFESSION,258,268,0


In [ ]:
df_tweets = df_tweets.drop(['entity', 'entity_tag', 'entity_pos_start_rel', 'entity_pos_end_rel'], axis=1)

In [ ]:
df_tweets = df_tweets.drop(df_tweets.tail(37).index)

In [ ]:
df_tweets.shape

(6600, 2)

In [ ]:
# создаем токенайзер для модели BERT, для его инициализации достаточно указать словарь, на котором обучалась предобученная модель
# BERT использует собственную токенизацию, никакой предобработки 

# tokenizer = ppb.BertTokenizer(vocab_file='vocab.txt')
from transformers import AutoTokenizer, AutoModel


tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
# токенизируем текст каждого твита, для BERT не требуется никакая дополнительная предобработка, лемматизация и прочее

tokenized = df_tweets['sentence'].astype(str).str[:512].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )

In [ ]:
df_tweets['sentence'].astype(str).str[:512]

0       Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...
1       «За всю нашу долгую карьеру нам довелось играт...
2       Ранее, 7 декабря, толпа болельщиков перекрыла ...
3       В субботу, 21 июля 2018 года, на арене СК «Оли...
4       Представитель талибов Забиулла Муджахид в твит...
                              ...                        
6595    В заявлении британского правительства говоритс...
6596    Накануне вечером около 150 тыс. человек вышли ...
6597    Высший руководитель Ирана, великий аятолла Али...
6598    В заявку сборной России под руководством Влади...
6599    Он был арестован на территории Великобритании ...
Name: sentence, Length: 6600, dtype: object

In [ ]:
# Пример токенизации текста: на входе - текст, а на выходе имеем массив с номерами токенов из словаря модели BERT

print(df_tweets['sentence'][0])
print(tokenized[0])
print(tokenizer.tokenize(df_tweets['sentence'][0]))

Джеймс «Бадди» Макгирт (James (Buddy) McGirt, тренер Дадашева упрашивал дагестанского спортсмена остановить бой, но тот хотел продолжать.
[101, 18688, 304, 91380, 326, 114499, 21193, 120, 11412, 120, 40720, 122, 104291, 83368, 128, 10109, 82922, 83448, 19398, 20944, 11817, 113837, 29186, 24548, 11519, 128, 3435, 8470, 17200, 26285, 132, 102]
['Джеймс', '«', 'Бадди', '»', 'Макги', '##рт', '(', 'James', '(', 'Buddy', ')', 'McG', '##irt', ',', 'тренер', 'Дада', '##шева', 'упр', '##аши', '##вал', 'дагестанского', 'спортсмена', 'остановить', 'бой', ',', 'но', 'тот', 'хотел', 'продолжать', '.']


In [ ]:
model_rubert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# инициализируем предобученную модель RuBERT из файла, 
# в json-файле конфигурации описаны параметры модели

# config = ppb.BertConfig.from_json_file('config.json')
# model = ppb.BertModel.from_pretrained('pytorch_model.bin', config = config, from_tf=True)

In [ ]:
model_rubert.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
# из-за того, что каждый твит в датасете имеет разную длину (количество токенов)
# мы делаем паддинг - заполнение нулями каждого массива токенов до длины максимального массива
# чтобы на выходе получить матрицу из токенизированных текстов одной длины

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
# посмотрим на размерность матрицы токенизированных твитов после паддинга

np.array(padded).shape

(6600, 172)

In [ ]:
# Накладываем маску на значимые токены
# В данном случае нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6600, 172)

In [ ]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import tqdm

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 100

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings = []

for i in tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to('cuda') 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
               last_hidden_states = model_rubert(input_ids, attention_mask=attention_mask_batch)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings.append(last_hidden_states[0][:,0,:].cpu().numpy())



100%|██████████| 66/66 [01:00<00:00,  1.09it/s]


In [ ]:
# преобразуем список батчей эмбеддингов в numpy-матрицу 
features = np.concatenate(embeddings)

In [ ]:
# выводим размерность полученной матрицы эмбеддингов
# данная модель BERT формирует вектора текстов в 768-мерном пространстве признаков
features.shape

(6600, 768)

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Импортируем необходимые библиотеки для обучения классификатора на logreg и оценки качества

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import scikitplot as skplt
import seaborn as sns

In [ ]:
# Сохраним целевую переменную: метку тональности позитив/негатив

labels = df_tweets['label']

In [ ]:
labels

0       0
1       0
2       0
3       0
4      -1
       ..
6595    1
6596    0
6597   -1
6598    1
6599    0
Name: label, Length: 6600, dtype: int64

In [ ]:
features.shape

(6600, 768)

In [ ]:
labels.shape

(6600,)

In [ ]:
# Разделяем матрицу признаков и целевую переменную на обучающий и тестовый набор

train_features, test_features, train_labels, test_labels = train_test_split(features, labels[:8000], test_size=0.1, random_state=42)

In [ ]:
catboost_params = {
    'iterations': 10000,
    'learning_rate': 0.001,
    'eval_metric': 'AUC',
    #'eval_metric': 'Accuracy',
    'loss_function': 'MultiClass',
    'task_type': 'GPU',
    'early_stopping_rounds': 1000,
    'use_best_model': True,
    'objective':"MultiClass",
    'verbose': 100
}

In [ ]:
X_tr = pd.DataFrame(train_features) #X_train
X_val =  pd.DataFrame(test_features)
y_tr = train_labels
y_val = test_labels

train_pool = Pool(
    X_tr, 
    y_tr
)
valid_pool = Pool(
    X_val, 
    y_val
)

In [ ]:
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6216274	best: 0.6216274 (0)	total: 109ms	remaining: 18m 13s
100:	test: 0.7104957	best: 0.7131025 (47)	total: 4.12s	remaining: 6m 43s
200:	test: 0.7200818	best: 0.7200818 (200)	total: 5.78s	remaining: 4m 41s
300:	test: 0.7269745	best: 0.7269745 (300)	total: 7.43s	remaining: 3m 59s
400:	test: 0.7320476	best: 0.7320476 (400)	total: 9.09s	remaining: 3m 37s
500:	test: 0.7351324	best: 0.7358385 (475)	total: 10.8s	remaining: 3m 23s
600:	test: 0.7389636	best: 0.7389636 (600)	total: 12.4s	remaining: 3m 13s
700:	test: 0.7419071	best: 0.7419071 (700)	total: 16.7s	remaining: 3m 41s
800:	test: 0.7442096	best: 0.7442858 (798)	total: 18.4s	remaining: 3m 31s
900:	test: 0.7468013	best: 0.7468744 (899)	total: 20.2s	remaining: 3m 23s
1000:	test: 0.7492293	best: 0.7492625 (996)	total: 21.9s	remaining: 3m 16s
1100:	test: 0.7511474	best: 0.7511474 (1100)	total: 23.6s	remaining: 3m 10s
1200:	test: 0.7531314	best: 0.7531314 (1200)	total: 25.3s	remaining: 3m 5s
1300:	test: 0.7544243	best: 0.7545418 

In [ ]:
import random

In [ ]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['sentence'][tweet_index])
print('Predict label: ', model.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['label'][tweet_index])


Text: 25 июня 2013 года президент Чехии Милош Земан предпочел назвать имя своего кандидата в премьер-министры.
Predict label:  [0]
True label:  0


In [ ]:
# оцениваем accuracy на тестовой выборке

model.score(test_features, test_labels)

0.7196969696969697

In [ ]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['sentence'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': model.predict(features[idx:idx+1][:])[0][0],
                'gtrue': df_tweets['label'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    

100%|██████████| 4000/4000 [00:37<00:00, 107.30it/s]


In [ ]:
df_train

,text,predict,gtrue
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",0,0
1,«За всю нашу долгую карьеру нам довелось играт...,0,0
2,"Ранее, 7 декабря, толпа болельщиков перекрыла ...",0,0
3,"В субботу, 21 июля 2018 года, на арене СК «Оли...",0,0
4,Представитель талибов Забиулла Муджахид в твит...,0,-1
...,...,...,...
3995,"В знак протеста против поведения единороссов, ...",0,0
3996,Дейенерис Таргариен и Джон Сноу на драконах не...,0,-1
3997,В Израиле отмечают Йом Ха-Зикарон - День памят...,0,0
3998,"Кроме того, при атаке были ранены пятеро военн...",0,0


In [ ]:
df_train.predict.value_counts()

 0    3718
-1     187
 1      95
Name: predict, dtype: int64

In [ ]:
df_train.gtrue.value_counts()

 0    2867
-1     616
 1     517
Name: gtrue, dtype: int64

In [ ]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [ ]:
confusion_matrix(df_train,'predict','gtrue')

gtrue,-1,0,1
predict,,,
-1,150,25,12
0,465,2830,423
1,1,12,82


In [ ]:
# обучаем классификатор на основе логистической регрессии

lr_clf = LogisticRegression(random_state=42, class_weight='balanced')
lr_clf.fit(train_features, train_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [ ]:
import random

In [ ]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['sentence'][tweet_index])
print('Predict label: ', lr_clf.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['label'][tweet_index])


Text: В Венесуэла|Венесуэле 7 октября 2012 года проходят Президентские выборы в Венесуэле (2012)|президентские выборы, которые, по мнению аналитиков, станут самыми напряженными за всю политическую карьеру действующего лидера страны Чавес, Уго|Уго Чавеса, пробывшего у власти почти 14 лет.
Predict label:  0
True label:  0


In [ ]:
# оцениваем accuracy на тестовой выборке

lr_clf.score(test_features, test_labels)

0.5333333333333333

In [ ]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['sentence'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': lr_clf.predict(features[idx:idx+1][:])[0],
                'gtrue': df_tweets['label'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    
    

100%|██████████| 4000/4000 [00:01<00:00, 2966.93it/s]


In [ ]:
df_train.predict.value_counts()

 0    1876
-1    1138
 1     986
Name: predict, dtype: int64

In [ ]:
df_train.gtrue.value_counts()

 0    2867
-1     616
 1     517
Name: gtrue, dtype: int64

In [ ]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [ ]:
confusion_matrix(df_train,'predict','gtrue')

gtrue,-1,0,1
predict,,,
-1,455,627,56
0,91,1692,93
1,70,548,368


In [ ]:
final_data = pd.read_csv('fin_data2.csv', sep='\t')

In [ ]:
final_data.head()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
0,Абдул Реза Шайхулислами выразил готовность Ира...,Абдул Реза Шайхулислами,PERSON,0,23
1,Абдул Реза Шайхулислами выразил готовность Ира...,афганцев,NATIONALITY,157,165
2,Абдул Реза Шайхулислами выразил готовность Ира...,правительству Афганистана,ORGANIZATION,68,93
3,В своем выступлении Абдул Реза Шайхулислами от...,Абдул Реза Шайхулислами,PERSON,20,43
4,"В свою очередь, Зорар Ахмад Мокбел заявил о то...",Ирана,COUNTRY,86,91


In [ ]:
final_data.shape

(1947, 5)

In [ ]:
final_data.tail()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
1942,От дальнейших комментариев Брукер воздержалась...,Брукер,PERSON,27,33
1943,"По данным Associated Press, в первый день 2011...",Associated Press,ORGANIZATION,10,26
1944,"По данным Associated Press, в первый день 2011...",Microsoft,ORGANIZATION,62,71
1945,"По данным на конец 2010 года, напоминает AP, у...",AP,ORGANIZATION,41,43
1946,Служба техподдержки Windows Live в ответ на по...,Служба техподдержки Windows Live,ORGANIZATION,0,32


In [ ]:
tokenized_fin = final_data['sentence'].astype(str).str[:512].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )
padded_fin = np.array([i + [0]*(max_len-len(i)) for i in tokenized_fin.values])

In [ ]:
attention_mask_f = np.where(padded_fin != 0, 1, 0)
attention_mask_f.shape

(1947, 172)

In [ ]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import tqdm

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 59

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings_fin = []

for i in tqdm(range(padded_fin.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids_f = torch.LongTensor(padded_fin[batch_size*i:batch_size*(i+1)]).to('cuda') 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch_f = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
               last_hidden_states_f = model_rubert(input_ids_f, attention_mask=attention_mask_batch_f)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings_fin.append(last_hidden_states_f[0][:,0,:].cpu().numpy())


100%|██████████| 33/33 [00:16<00:00,  2.02it/s]


In [ ]:
features_fin = np.concatenate(embeddings_fin)

In [ ]:
predictions_lr_clf = lr_clf.predict(features_fin)
predictions_model = model.predict(features_fin)

In [ ]:
print(predictions_model)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
np.unique(predictions_lr_clf)

array([-1,  0,  1])

In [ ]:
pd.DataFrame(predictions_model).to_csv('catboost_rubert.csv')

In [ ]:
lr_clf_pred = pd.DataFrame(predictions_lr_clf)

In [ ]:
lr_clf_pred.head()

,0
0,-1
1,0
2,-1
3,0
4,0


In [ ]:
lr_clf_pred.rename(columns={0: "ans"})

,ans
0,-1
1,0
2,-1
3,0
4,0
...,...
1942,0
1943,0
1944,-1
1945,0


In [ ]:
lr_clf_pred.to_csv('ans_lr_clf.csv', index=False, header=False)

In [ ]:
new_pred = lr_clf_pred.astype(int)

In [ ]:
new_pred.to_csv('ans.csv', columns=[0])